---
title: "1. Feature Exploration"
date: 2021-02-24
type: technical_note
draft: false
---

## HSFS feature exploration

In this notebook we are going to walk through how to use the HSFS library to explore feature groups and features in the Hopsworks Feature Store. 

A key component of the Hopsworks feature store is to enable sharing and re-using of features across models and use cases. As such, the HSFS libraries allows user to join features from different feature groups and use them to create training datasets.
Features can be taken also from different feature stores (projects) as long as the user running the notebook has the read access to those.

![Join](../images/join.svg "Join")

As for the [feature_engineering](./feature_engineering.ipynb) notebook, the first step is to establish a connection with the feature store and retrieve the project feature store handle.

In [1]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
10,application_1652868601260_0018,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

### Explore feature groups

You can interact with the feature groups as if they were Spark dataframe. A feature group object has a `show()` method, to show `n` number of lines, and a `read()` method to read the content of the feature group in a Spark dataframe.

The first step to do any operation on a feature group is to get its handle from the feature store. The `get_feature_group` method accepts the name of the feature group and an optional parameter with the version you want to select. If you do not provide a version, the APIs will default to version 1

In [2]:
sales_fg = fs.get_feature_group("sales_fg", version = 1)

In [3]:
sales_fg.show(5)

+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+--------------------------+---------------------+
|store|dept|      date|weekly_sales|is_holiday|sales_last_month_store_dep|sales_last_quarter_store_dep|sales_last_six_month_store_dep|sales_last_year_store_dep|sales_last_month_store|sales_last_quarter_store|sales_last_six_month_store|sales_last_year_store|
+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+--------------------------+---------------------+
|   31|  47|2010-02-05|        86.0|     false|                       0.0|                         0.0|                           0.0|                      0.0|                   0.0|                     0.0|                  

In [4]:
sales_df = sales_fg.read()
sales_df.filter("store == 20").show(5)
print(type(sales_df))

+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+--------------------------+---------------------+
|store|dept|      date|weekly_sales|is_holiday|sales_last_month_store_dep|sales_last_quarter_store_dep|sales_last_six_month_store_dep|sales_last_year_store_dep|sales_last_month_store|sales_last_quarter_store|sales_last_six_month_store|sales_last_year_store|
+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+--------------------------+---------------------+
|   20|  55|2010-02-05|    32362.95|     false|                       0.0|                         0.0|                           0.0|                      0.0|                   0.0|                     0.0|                  

You can also inspect the metadata of the feature group. You can, for instance, show the features the feature group is made of and if they are primary or partition keys:

In [5]:
print("Name: {}".format(sales_fg.name))
print("Description: {}".format(sales_fg.description))
print("Features:")
features = sales_fg.features
for feature in features:
    print("{:<60} \t Primary: {} \t Partition: {}".format(feature.name, feature.primary, feature.partition))

Name: sales_fg
Description: Sales related features
Features:
store                                                        	 Primary: True 	 Partition: False
dept                                                         	 Primary: True 	 Partition: False
date                                                         	 Primary: True 	 Partition: False
weekly_sales                                                 	 Primary: False 	 Partition: False
is_holiday                                                   	 Primary: False 	 Partition: False
sales_last_month_store_dep                                   	 Primary: False 	 Partition: False
sales_last_quarter_store_dep                                 	 Primary: False 	 Partition: False
sales_last_six_month_store_dep                               	 Primary: False 	 Partition: False
sales_last_year_store_dep                                    	 Primary: False 	 Partition: False
sales_last_month_store                                       	 Primar

If you are interested only in a subset of features, you can use the `select()` method on the feature group object to select a list of features. The `select()` behaves like a feature group, as such, you can call the `.show()` or `.read()` methods on it.

In [6]:
sales_fg.select(['store', 'dept', 'weekly_sales']).show(5)

+-----+----+------------+
|store|dept|weekly_sales|
+-----+----+------------+
|   31|  47|        86.0|
|   31|   7|    13422.33|
|   31|  42|     4596.97|
|   31|  34|    19443.48|
|   31|  36|       434.0|
+-----+----+------------+
only showing top 5 rows

If your feature group is available both online and offline, you can use the `online` option of the `show()` and `read()` methods to specify if you want to read your feature group from online storage.

In [7]:
sales_fg_3 = fs.get_feature_group('sales_fg', 3)

sales_fg_3.select(['store', 'dept', 'weekly_sales']).show(5, online=True)

+-----+----+------------+
|store|dept|weekly_sales|
+-----+----+------------+
|   34|   8|    29764.29|
|   34|   7|    12513.47|
|   34|  31|     1125.34|
|   34|  67|    14844.98|
|   34|  35|      2214.0|
+-----+----+------------+
only showing top 5 rows

### Filter Feature Groups

If you do not want to read your feature group into a dataframe, you can also filter directly on a `FeatureGroup` object. Applying a filter to a feature group returns a `Query` object which can subsequently be used to be further joined with other feature groups or queries, or it can be materialized as a training dataset.

In [8]:
sales_fg.select(['store', 'dept', 'weekly_sales']).filter(sales_fg.weekly_sales >= 50000).show(5)

+-----+----+------------+
|store|dept|weekly_sales|
+-----+----+------------+
|   31|  90|    89807.88|
|   31|  94|    61298.99|
|   31|  40|    59656.92|
|   31|  93|    50856.18|
|   31|  72|    87869.15|
+-----+----+------------+
only showing top 5 rows

Conjunctions of filters can be constructed using the Python Bitwise Operators `|` and `&` which replace the regular binary operators when working with feature groups and filters.

In [9]:
sales_fg.select(['store', 'dept', 'weekly_sales']).filter((sales_fg.weekly_sales >= 50000) & (sales_fg.dept == 2)).show(5)

+-----+----+------------+
|store|dept|weekly_sales|
+-----+----+------------+
|   31|   2|    58982.08|
|   31|   2|    56830.82|
|   31|   2|    58232.69|
|   31|   2|    56767.26|
|   31|   2|    57112.57|
+-----+----+------------+
only showing top 5 rows

### Join Features and Feature Groups

HSFS provides an API similar to Pandas to join feature groups together and to select features from different feature groups.
The easies query you can write is by selecting all the features from a feature group and join them with all the features of another feature group.

You can use the `select_all()` method of a feature group to select all its features. HSFS relies on the Hopsworks feature store to identify which features of the two feature groups to use as joining condition. 
If you don't specify anything, Hopsworks will use the largest matching subset of primary keys with the same name.

In the example below, `sales_fg` has `store`, `dept` and `date` as composite primary key while `exogenous_fg` has only `store` and `date`. So Hopsworks will set as joining condition `store` and `date`.

In [10]:
sales_fg = fs.get_feature_group('sales_fg', version = 1)
exogenous_fg = fs.get_feature_group('exogenous_fg', version = 1)

query = sales_fg.select_all().join(exogenous_fg.select_all())

You can use the query object to create training datasets (see training dataset notebook). You can inspect the query generated by calling the `to_string()` method on it.

In [11]:
print(query.to_string())

SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`date` `date`, `fg1`.`weekly_sales` `weekly_sales`, `fg1`.`is_holiday` `is_holiday`, `fg1`.`sales_last_month_store_dep` `sales_last_month_store_dep`, `fg1`.`sales_last_quarter_store_dep` `sales_last_quarter_store_dep`, `fg1`.`sales_last_six_month_store_dep` `sales_last_six_month_store_dep`, `fg1`.`sales_last_year_store_dep` `sales_last_year_store_dep`, `fg1`.`sales_last_month_store` `sales_last_month_store`, `fg1`.`sales_last_quarter_store` `sales_last_quarter_store`, `fg1`.`sales_last_six_month_store` `sales_last_six_month_store`, `fg1`.`sales_last_year_store` `sales_last_year_store`, `fg0`.`temperature` `temperature`, `fg0`.`fuel_price` `fuel_price`, `fg0`.`markdown1` `markdown1`, `fg0`.`markdown2` `markdown2`, `fg0`.`markdown3` `markdown3`, `fg0`.`markdown4` `markdown4`, `fg0`.`markdown5` `markdown5`, `fg0`.`cpi` `cpi`, `fg0`.`unemployment` `unemployment`, `fg0`.`is_holiday` `is_holiday`, CASE WHEN `fg0`.`appended_feature` IS 

As for the feature groups, you can call the `show()` method to inspect the data before generating a training dataset from it. Or you can call the `read()` method to get a Spark DataFrame with the result of the query and apply additional transformations to it.

In [12]:
query.show(5)

+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+--------------------------+---------------------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+----------+----------------+
|store|dept|      date|weekly_sales|is_holiday|sales_last_month_store_dep|sales_last_quarter_store_dep|sales_last_six_month_store_dep|sales_last_year_store_dep|sales_last_month_store|sales_last_quarter_store|sales_last_six_month_store|sales_last_year_store|temperature|fuel_price|markdown1|markdown2|markdown3|markdown4|markdown5|        cpi|unemployment|is_holiday|appended_feature|
+-----+----+----------+------------+----------+--------------------------+----------------------------+------------------------------+-------------------------+----------------------+------------------------+------------------------

As for the `show()` and `read()` method of the feature group, even in the case of a query you can specify against which storage to run the query.

### Select only a subset of features

You can replace the `select_all()` method with the `select([])` method to be able to select only a subset of features from a feature group you want to join:

In [13]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']))
query.show(5)

+-----+----+------------+----------+
|store|dept|weekly_sales|fuel_price|
+-----+----+------------+----------+
|   31|  47|        86.0|     2.572|
|   31|   7|    13422.33|     2.572|
|   31|  42|     4596.97|     2.572|
|   31|  34|    19443.48|     2.572|
|   31|  36|       434.0|     2.572|
+-----+----+------------+----------+
only showing top 5 rows

### Overwrite the joining key

If your feature groups don't have a primary key, or if they have different names or if you want to overwrite the joining key, you can pass it as a parameter of the join.

As in Pandas, if the feature has the same name on both feature groups, then you can use the `on=[]` paramter. If they have different names, then you can use the `left_on=[]` and `right_on=[]` paramters:

In [14]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']), on=['date'])
query.show(5)

+-----+----+------------+----------+
|store|dept|weekly_sales|fuel_price|
+-----+----+------------+----------+
|   31|  47|        86.0|     2.784|
|   31|  47|        86.0|     2.666|
|   31|  47|        86.0|     2.572|
|   31|  47|        86.0|     2.962|
|   31|  47|        86.0|      2.58|
+-----+----+------------+----------+
only showing top 5 rows

### Overwriting the join type

By default, the join type between two feature groups is `INNER JOIN`. You can overwrite this behavior by passing the `join_type` parameter to the join method. Valid types are: `INNER, LEFT, RIGHT, FULL, CROSS, LEFT_SEMI_JOIN, COMMA`

In [15]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']), join_type="left")

print(query.to_string())

SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`weekly_sales` `weekly_sales`, `fg0`.`fuel_price` `fuel_price`
FROM `hsfs_basics_featurestore`.`sales_fg_1` `fg1`
LEFT JOIN `hsfs_basics_featurestore`.`exogenous_fg_1` `fg0` ON `fg1`.`store` = `fg0`.`store` AND `fg1`.`date` = `fg0`.`date`

### Join mulitple feature groups

You can concatenate as many feature gropus as you wish. In the example below the order of execution will be:

    (`sales_fg` <> `store_fg`) <> `exogenous_fg`

The join paramers you pass in each `join()` method call apply to that specific join. This means that you can concatenate left and right joins.
Please be aware that currently HSFS **does not support** nested join such as: 

    `sales_fg` <> (`store_fg` <> `exogenous_fg`)

In [18]:
store_fg = fs.get_feature_group("store_fg",version = 2)

query = sales_fg.select_all()\
                .join(store_fg.select_all())\
                .join(exogenous_fg.select(['fuel_price', 'unemployment', 'cpi']))

print(query.to_string())

SELECT `fg2`.`store` `store`, `fg2`.`dept` `dept`, `fg2`.`date` `date`, `fg2`.`weekly_sales` `weekly_sales`, `fg2`.`is_holiday` `is_holiday`, `fg2`.`sales_last_month_store_dep` `sales_last_month_store_dep`, `fg2`.`sales_last_quarter_store_dep` `sales_last_quarter_store_dep`, `fg2`.`sales_last_six_month_store_dep` `sales_last_six_month_store_dep`, `fg2`.`sales_last_year_store_dep` `sales_last_year_store_dep`, `fg2`.`sales_last_month_store` `sales_last_month_store`, `fg2`.`sales_last_quarter_store` `sales_last_quarter_store`, `fg2`.`sales_last_six_month_store` `sales_last_six_month_store`, `fg2`.`sales_last_year_store` `sales_last_year_store`, `fg0`.`type` `type`, `fg0`.`size` `size`, `fg0`.`num_depts` `num_depts`, `fg1`.`fuel_price` `fuel_price`, `fg1`.`unemployment` `unemployment`, `fg1`.`cpi` `cpi`
FROM `hsfs_basics_featurestore`.`sales_fg_1` `fg2`
INNER JOIN `hsfs_basics_featurestore`.`store_fg_2` `fg0` ON `fg2`.`store` = `fg0`.`store`
INNER JOIN `hsfs_basics_featurestore`.`exogenous

### Use Joins together with Filters

It is possible to use filters in any of the subqueries of a joined query.

In [19]:
query = sales_fg.select_all()\
                .join(store_fg.select_all())\
                .join(exogenous_fg.select(['fuel_price', 'unemployment', 'cpi']).filter(exogenous_fg.fuel_price <= 2.7)) \
                .filter(sales_fg.weekly_sales >= 50000)

print(query.to_string())

SELECT `fg2`.`store` `store`, `fg2`.`dept` `dept`, `fg2`.`date` `date`, `fg2`.`weekly_sales` `weekly_sales`, `fg2`.`is_holiday` `is_holiday`, `fg2`.`sales_last_month_store_dep` `sales_last_month_store_dep`, `fg2`.`sales_last_quarter_store_dep` `sales_last_quarter_store_dep`, `fg2`.`sales_last_six_month_store_dep` `sales_last_six_month_store_dep`, `fg2`.`sales_last_year_store_dep` `sales_last_year_store_dep`, `fg2`.`sales_last_month_store` `sales_last_month_store`, `fg2`.`sales_last_quarter_store` `sales_last_quarter_store`, `fg2`.`sales_last_six_month_store` `sales_last_six_month_store`, `fg2`.`sales_last_year_store` `sales_last_year_store`, `fg0`.`type` `type`, `fg0`.`size` `size`, `fg0`.`num_depts` `num_depts`, `fg1`.`fuel_price` `fuel_price`, `fg1`.`unemployment` `unemployment`, `fg1`.`cpi` `cpi`
FROM `hsfs_basics_featurestore`.`sales_fg_1` `fg2`
INNER JOIN `hsfs_basics_featurestore`.`store_fg_2` `fg0` ON `fg2`.`store` = `fg0`.`store`
INNER JOIN `hsfs_basics_featurestore`.`exogenous

### Free hand query

With HSFS you are free of writing skipping entirely the Hopsworks query constructor and write your own query. This functionality can be useful if you need to express more complex queries for your use case. `fs.sql` returns a Spark Dataframe.

In [20]:
fs.sql("SELECT * FROM `store_fg_1`")

DataFrame[store: int, type: string, size: int, num_depts: bigint]

---